<a href="https://colab.research.google.com/github/Anze-/datathon2k25/blob/alberto/feature_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **OrderFox Hackathon: Domain-Specific Chatbot Design** 🚀  

## **📌 Overview**  
In this hackathon, your goal is to design and implement a domain-specific chatbot using **Retrieval-Augmented Generation (RAG)**. You will be provided with a **dataset of HTML-crawled documents** in jason format, and your task is to build a system that effectively retrieves relevant information and generates accurate responses.  

## **🔹 What You Need to Do**  
1. **Set up your own environment** - A github Repository, or make a copy of this colab notebook, or a Kaggle notebook.
2. **Load and Process Documents** – Extract text from the json files provided to you.
3. **Implement Document Retrieval**
– Try different retrieval approaches, including:  
   - Keyword-based search  
   - Vector embeddings (e.g. Bag-of-Words, Word2Vec, embedding models on Hugging Face, embeddig models provided with OpenAI API)  + vector database
   - Graph / Tree extraction + graph database
   - Hybrid methods   
4. **Perform Response Generation** – Retrieve relevant documents and generate responses using a Language Model:
  - Feel free to explore prompt engineer

5. **Optimize and Evaluate Your System** – Compare performance based on relevance, grounding, fluency, efficiency, and cost.  
6. **Deliver a Well-Structured Solution** – Organize your code as a modular project repository.  
 - Code repository or notebook
 - Make sure to include your knowledge database

## **📦 What Is Provided?**  
✅ **Dataset**: JSON files containing extracted HTML content, available on a shared Google Drive and Kaggle.  
✅ **Baseline Notebook**: A starter kit with useful tools and guidance.  
✅ **Evaluation Metrics**: A structured evaluation framework to assess performance.  

## **🖥️ What Computing Resources Can You Use?**  
You are welcome to choose your preferred platform to develop your solution. Either   
- Locally (on your own machine) 💻  
- or Using **cloud platforms** such as **Google Colab** or **Kaggle** (a free account is sufficient).  

## **🛠️ Tools You May Consider**  
(*These are recommendations to help you get started. You are free to use alternative tools—just document your choices clearly!*)  
- **Database**: FAISS, ChromaDB, SQLite, Elasticsearch, Neo4j and etc.  
- **Embedding Models**: Hugging Face Sentence-Transformers, OpenAI Embeddings  
- **LLM for Generation**: OpenAI: gpt-4o-mini
- **Others**: Langchain, GraphRAG, and etc.

## **📌 Final Delivery**  
Your final submission should include:  
✅ A well-documented **GitHub repository or notebook**  
✅ A clear **README** explaining your approach  
✅ A structured **retrieval and generation modules**  

### **🔥 Bonus Points For**  
✨ Innovative retrieval techniques  
✨ Well-organized, modular code  
✨ Creative visualizations or user interfaces  


# 1. Set up working environment

In [6]:
!pip install openai

# Database options
!pip install chromadb # if you use chromadb as your vector database

# Others
!pip install langchain-community # if you use langchain for orchastration
!pip install transformers #if you use huggingface for vector embedding

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8): No such file or directory
error: externally-managed-environment

× This environment is externally managed
╰─> 
    The system-wide Python installation in Gentoo should be maintained
    using the system package manager (e.g. emerge).
    
    If the package in question is not packaged for Gentoo, please
    consider installing it inside a virtual environment, e.g.:
    
    python -m venv /path/to/venv
    . /path/to/venv/bin/activate
    pip install mypackage
    
    To exit the virtual environment, run:
    
    deactivate
    
    The virtual environment is not deleted, and can be re-entered by
    re-sourcing the activate file.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detailed specification.
/bin/bash

In [7]:
# enable GPU if needed, GPU can speed up your vector embedding if you computing these vectors locally (not using API)

import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [8]:

import os
import json
import chromadb
import openai
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

# Set OpenAI API Key
os.environ["OPENAI_API_KEY"] = ""


# 2. Knowledge Base Preparation

## 2.1 Load documents

Once you are added access to this folder, it will appear at your google drive "Shared drives". Then you can mount your drive and as following, and access your data from "/content/drive/Shared drives/Datathon/Data/hackathon_data/". Enjoy the ride! :)

In [3]:
# Load the Drive and mount
# from google.colab import drive
# drive.mount('/content/drive/')

ModuleNotFoundError: No module named 'google.colab'

Load json file.

In [9]:
#folder_path = "/content/drive/Shared drives/Datathon/Data/hackathon_data/"# Google drive path of the dataset
folder_path = "data/hackathon_data"
files_in_folder = os.listdir(folder_path)

len(files_in_folder)

13144

In [4]:
def load_documents(json_file):
    """Loads the JSON file."""
    with open(json_file, 'r') as f:
      try:
          data = json.load(f)
          return data
      except json.JSONDecodeError:
          print(f"Error reading {json_file}, it may not be a valid JSON file.")
    return []



In [18]:
for filename in files_in_folder:
    if filename.endswith('.json'):
        if filename == "panoston.com.json":

            file_path = os.path.join(folder_path, filename)
            doc = load_documents(file_path)
            print(file_path)

print(doc["text_by_page_url"]["https://www.panoston.com/about/news/"])

data/hackathon_data/panoston.com.json
News - Pan-Oston
Skip to content
Menu
Contact Us:
(800) 210 – 2302
Pan-Oston News
Newsletter Signup
Get updates on new Pan-Oston products and offers by signing up for our newsletter.
Email
*
/* = 0;if(!is_postback){return;}var form_content = jQuery(this).contents().find(‘#gform_wrapper_1’);var is_confirmation = jQuery(this).contents().find(‘#gform_confirmation_wrapper_1’).length > 0;var is_redirect = contents.indexOf(‘gformRedirect(){‘) >= 0;var is_form = form_content.length > 0 && ! is_redirect && ! is_confirmation;var mt = parseInt(jQuery(‘html’).css(‘margin-top’), 10) + parseInt(jQuery(‘body’).css(‘margin-top’), 10) + 100;if(is_form){jQuery(‘#gform_wrapper_1’).html(form_content.html());if(form_content.hasClass(‘gform_validation_error’)){jQuery(‘#gform_wrapper_1’).addClass(‘gform_validation_error’);} else {jQuery(‘#gform_wrapper_1’).removeClass(‘gform_validation_error’);}setTimeout( function() { /* delay the scroll by 50 milliseconds to fix a bug

## 2.2 Pre-process documents.

Feel free to explore and pre-process the data. You may want to clean or segment the documents as you see fit.

In [7]:
def page_segment(docs):
    """You may prefer to load each page separately."""
    i = 0
    page_segment = []
    for s in list(docs['text_by_page_url'].values()):
      page_segment.append({"docID": docs['doc_id'], "pageID": 'page_' + str(i), "text": s})
      i += 1
    return page_segment

In [8]:
def segment_documents(docs, chunk_size=500):
    """Segments documents into chunks of a given token size. Replace this function with your segmentation approach or maybe use the original document without segmentation."""
    segmented = []
    for doc_id, content in docs.items():
        for i in range(0, len(content), chunk_size):
            segment = content[i : i + chunk_size]
            segmented.append({"id": doc_id, "text": segment})
    return segmented



In [9]:
def document_clean(docs):
  """
  You may want to clean the dataset, add the code here.
  """
  pass

## 2.3 Document Indexing and Storage (Profiling)

Feel free to choose different ways to indexing and storing the provided documents in a knowledge database.

So that they can be retrieved in different ways according to your system design choices, such as search by keywords, vector representation, graph relation, and etc.

# 3. Retrieval Augmented Generation

## 3.1 Load Knowledge Database

## 3.2 Relevant Document Retrieval

Feel free to check and improve your retrieval performance as it affect the generation results significantly.

In [ ]:
def retrieve_documents(query, db_path, embedding_model):
  """
  retrieve relevant documents from the knowledge database to the query.
  """
  return relevant_docs

## 3.3 Response Generation

Feel free to explore promp engineer to improve the quality of your generated response.

The retrieved documents are used as context to generate more relevant response. Gereral knowledge from the language model itself is also used.

In [ ]:
def generate_answer(query, retrieved_texts, prompt_template):
    """Generates an answer using retrieved documents and GPT-4."""
    return response

In [ ]:
query = "What company is located in 29010 Commerce Center Dr., Valencia, 91355, California, US?"
retrieved_docs = retrieve_documents(query, db_path, embedding_model)
response = generate_answer(query, retrieved_texts, prompt_template)

print("Query:", query)
print("Retrieved Documents:", [doc.page_content for doc in retrieved_docs])
print("Generated Answer:", response)

# 4. Evaluation

Try as many examples to evaluate your system and improve your performance!

As the final sysrtem will be evaluated from various aspects. Try to check different metrics when you evaluate. One trick is to do a "strict RAG" where the response is generated based on the retrieved documents only, i.e. no general knowledge from the LLMs will be used. This may be a good way to check if your retrieval part is working as expected. Note, that in the final system general knowledge from the LLMs are welcome. "Strict RAG" is only used as a way for you to check your performance :)